# Radar Plots

import libs and set default configurations

In [ ]:
import os
import numpy as np
from typing import List
import matplotlib
import matplotlib.pyplot as plt

from matplotlib.patches import Circle, RegularPolygon
from matplotlib.path import Path
from matplotlib.projections import register_projection
from matplotlib.projections.polar import PolarAxes
from matplotlib.spines import Spine
from matplotlib.transforms import Affine2D

plt.rc("font", family="Times New Roman", size=14)
plt.rcParams["lines.linewidth"] = 2
plt.rcParams["text.usetex"] = True
plt.rcParams["xtick.direction"] = "in"
plt.rcParams["ytick.direction"] = "in"
tick_params = {"direction": "in", "length": 4, "width": 1, "bottom": True, "top": True, "left": True, "right": True}
line_configs = {"linewidth": 2, "markersize": 12, "markeredgewidth": 1.5, "fillstyle": "none"}

prerequisite functions for drawing radar

In [ ]:
def rgb2hex(rgb):
    color = "#"
    for num in rgb:
        hex_num = hex(num)
        color += hex_num[2:].zfill(2)
    return color

def radar_factory(num_vars, frame="circle"):
    # calculate evenly-spaced axis angles
    theta = np.linspace(0, 2 * np.pi, num_vars, endpoint=False)

    class RadarTransform(PolarAxes.PolarTransform):

        def transform_path_non_affine(self, path):
            # Paths with non-unit interpolation steps correspond to gridlines,
            # in which case we force interpolation (to defeat PolarTransform's
            # autoconversion to circular arcs).
            if path._interpolation_steps > 1:
                path = path.interpolated(num_vars)
            return Path(self.transform(path.vertices), path.codes)

    class RadarAxes(PolarAxes):
        name = "radar"
        PolarTransform = RadarTransform

        def __init__(self, *args, **kwargs):
            super().__init__(*args, **kwargs)
            # rotate plot such that the first axis is at the top
            self.set_theta_zero_location('N')

        def fill(self, *args, closed=True, **kwargs):
            """Override fill so that line is closed by default"""
            return super().fill(closed=closed, *args, **kwargs)

        def plot(self, *args, **kwargs):
            """Override plot so that line is closed by default"""
            lines = super().plot(*args, **kwargs)
            for line in lines:
                self._close_line(line)

        def _close_line(self, line):
            x, y = line.get_data()
            # FIXME: markers at x[0], y[0] get doubled-up
            if x[0] != x[-1]:
                x = np.append(x, x[0])
                y = np.append(y, y[0])
                line.set_data(x, y)

        def set_varlabels(self, labels):
            self.set_thetagrids(np.degrees(theta), labels)

        def _gen_axes_patch(self):
            # The Axes patch must be centered at (0.5, 0.5) and of radius 0.5
            # in axes coordinates.
            if frame == "circle":
                return Circle((0.5, 0.5), 0.5)
            elif frame == "polygon":
                return RegularPolygon((0.5, 0.5), num_vars,
                                      radius=.5, edgecolor="k")
            else:
                raise ValueError("Unknown value for 'frame': %s" % frame)

        def _get_axes_spines(self):
            if frame == "circle":
                return super()._gen_axes_spines()
            elif frame == "polygon":
                # spine_type must be 'left'/'right'/'top'/'bottom'/'circle'.
                spine = Spine(axes=self,
                              spine_type="circle",
                              path=Path.unit_regular_polygon(num_vars))
                # unit_regular_polygon gives a polygon of radius 1 centered at
                # (0, 0) but we want a polygon of radius 0.5 centered at (0.5,
                # 0.5) in axes coordinates.
                spine.set_transform(Affine2D().scale(.5).translate(.5, .5)
                                    + self.transAxes)
                return {'polar': spine}
            else:
                raise ValueError("Unknown value for 'frame': %s" % frame)

    register_projection(RadarAxes)
    return theta

function to draw a 3*3 radar plot

In [ ]:
def draw_three_dataset_radars(res1, res2, res3, datasets, plot_names, labels, num_feats: int, dimension: List[str], save_dir, exp_name):
    theta = radar_factory(num_feats, frame="polygon")
    total_colors = [rgb2hex([75, 101, 175]), rgb2hex([127, 203, 164]), rgb2hex([244, 111, 68]), rgb2hex([164, 5, 69])]
    fig, axs = plt.subplots(figsize=(9, 9), nrows=3, ncols=3,
                            subplot_kw=dict(projection="radar"))
    fig.subplots_adjust(wspace=0.6, hspace=-0.2, top=0.95, bottom=0.0)
    artists = []
    for idx, ax in enumerate(axs.flat):
        if idx < 3:
            res = res1
            ax.set_rgrids([0.2, 0.4, 0.6, 0.8], angle=72)
        elif idx < 6:
            res = res2
            ax.set_rgrids([0.1, 0.3, 0.5, 0.7, 0.9], angle=72)
        else:
            res = res3
            ax.set_rgrids([0.1, 0.3, 0.5, 0.7, 0.9], angle=72)
        data = res[idx % 3]
        data_name = plot_names[idx % 3] + "\n" + datasets[idx // 3]
        colors = [total_colors[0], total_colors[idx % 3 + 1]]
        ax.set_title(data_name, y=-0.35)
        for line_id, (d, color) in enumerate(zip(data, colors)):
            if line_id == 0:
                ax.plot(theta, d, color=color, linewidth=1.5, label=labels[0])
            else:
                ax.plot(theta, d, color=color, linewidth=1.5, label=labels[idx // 3 + 1])
            ax.fill(theta, d, facecolor=color, alpha=0.5)
        # Go through labels and adjust alignment based on where
        # it is in the circle.
        for label, angle in zip(ax.get_xticklabels(), theta):
            if angle in (0, np.pi):
                label.set_horizontalalignment("center")
            elif 0 < angle < np.pi:
                label.set_horizontalalignment("right")
            else:
                label.set_horizontalalignment("left")
        ax.set_varlabels(dimension)
        ax.spines["polar"].set_linewidth(1.5)
        ax.grid(linestyle="--")
    for idx, ax in enumerate(fig.axes):
        if idx % 3 == 0:
            if idx == 0:
                artists.extend(ax.get_legend_handles_labels()[0])
            else:
                artists.append(ax.get_legend_handles_labels()[0][-1])
        else:
            continue
    fig.legend(artists, labels, loc="upper center", bbox_to_anchor=(0.52, 0.94), ncol=4, fontsize=14, columnspacing=0.2, frameon=False)

    save_path = os.path.join(save_dir, exp_name + ".png")
    fig.savefig(save_path, dpi=300)
    plt.show()

prepare data and draw a plot

In [ ]:
save_dir = "../figure"
dimensions = ["A", "B", "C", "D", "E"]
dataset_names = ["dataset 1", "dataset 2", "dataset 3"]
plot_names = ["fig 1 in the group", "fig 2 in the group", "fig 3 in the group"]
labels = ["method i", "method ii", "method iii", "method iv"]
group1 = [
    [[0.865131579, 0.38, 0.54, 0.84407, 0.54534], [0.8126, 0.32, 0.52, 0.84253, 0.54291]],
    [[0.865131579, 0.38, 0.54, 0.84407, 0.54534], [0.8118, 0.32, 0.52, 0.8426, 0.54233]],
    [[0.865131579, 0.38, 0.54, 0.84407, 0.54534], [0.8118, 0.21, 0.61, 0.8426, 0.54233]]
]
group2 = [
    [[0.95, 0.94, 0.85, 0.85536, 0.25672], [0.94, 0.92, 0.8, 0.85596, 0.25658]],
    [[0.95, 0.94, 0.85, 0.85536, 0.25672], [0.94, 0.92, 0.8, 0.856, 0.2566]],
    [[0.95, 0.94, 0.85, 0.85536, 0.25672], [0.94, 0.92, 0.8, 0.856, 0.25696]]
]
group3 = [
    [[1., 0.31, 0.72, 0.69304, 0.3972], [1., 0.3, 0.72, 0.68741, 0.39371]],
    [[1., 0.31, 0.72, 0.69304, 0.3972], [1., 0.3, 0.72, 0.68741, 0.39371]],
    [[1., 0.31, 0.72, 0.69304, 0.3972], [1., 0.31, 0.79, 0.68741, 0.39371]]
]

draw_three_dataset_radars(
    group1, group2, group3,
    dataset_names, plot_names, labels, len(dimensions), dimensions,
    save_dir, "radar_plot"
)